In [1]:
import json
import pandas as pd
import numpy as np
from ipywidgets import widgets
from numpy.linalg import norm
from neo4j import GraphDatabase

In [2]:
with open('config.json', 'r') as f:
    credentials = json.load(f)

uri = credentials['uri']
user = credentials['user']
password = credentials['password']
driver = GraphDatabase.driver(uri, auth=(user, password))
#session = driver.session()

In [3]:
def get_options(driver):
    session = driver.session()
    result = []
    query = """
            MATCH (u:User)-[r]-(n:Symptom)
            RETURN DISTINCT n.symptom AS symptom, COUNT(u) AS ocurrency
            ORDER BY ocurrency DESC
        """
    with session as s:
        tx = s.begin_transaction()
        for row in tx.run(query):
            try:
                if row:
                    symptom = (row[0]).strip()
                    ocurrency = row[1]
                    node_dict = {"symptom": symptom, "ocurrency": ocurrency}
                    result.append(node_dict)
            except Exception as e:
                print(e, row)
        tx.commit()
    return result

In [5]:
options = get_options(driver)

In [11]:
options_list = [x['symptom'] for x in options]
options_list_top_10 = []
for i in range(0, 20): 
    options_list_top_10.append(options_list[i])

In [13]:
len(options_list_top_10)

20

In [17]:
widget_seleccion = widgets.SelectMultiple(
    options=options_list_top_10,
    value=[options_list_top_10[0]],
    description='Selecciona opciones:'
)

widget_seleccion.layout.height = '200px'
display(widget_seleccion)


SelectMultiple(description='Selecciona opciones:', index=(0,), layout=Layout(height='200px'), options=('fatigu…

In [22]:
query = """
        WITH {0} AS new_symptoms
        MATCH (p:Patient)-[s:SIMILAR]-(o:Patient)
        WHERE ANY(symptom IN p.symptoms WHERE symptom IN new_symptoms)
        RETURN DISTINCT o.disease AS disease, s.score AS similarity
        ORDER BY similarity DESC
    """
query = query.format([x for x in widget_seleccion.value])
print(query)


        WITH ['high fever', 'headache', 'yellowing of eyes'] AS new_symptoms
        MATCH (p:Patient)-[s:SIMILAR]-(o:Patient)
        WHERE ALL(symptom IN p.symptoms WHERE symptom IN new_symptoms)
        RETURN DISTINCT o.disease AS disease, s.score AS similarity
        ORDER BY similarity DESC
    


In [28]:
def diagnose(driver,symptoms):
    possible_diseases = []
    session = driver.session()
    query = """
        WITH {0} AS new_symptoms
        MATCH (p:Patient)-[s:SIMILAR]-(o:Patient)
        WHERE ANY(symptom IN p.symptoms WHERE symptom IN new_symptoms)
        RETURN DISTINCT o.disease AS disease, s.score AS similarity
        ORDER BY similarity DESC
        LIMIT 20
    """
    query = query.format([x for x in symptoms])
    with driver.session() as session:
        tx = session.begin_transaction()
        for row in tx.run(query):
            try:
                if row:
                    disease = (row[0]).strip()
                    similarity = row[1]
                    node_dict = {"disease": disease, "similarity": similarity}
                    possible_diseases.append(node_dict)
            except Exception as e:
                print(e, row)
        tx.commit()
    return possible_diseases
    

In [29]:
result = diagnose(driver=driver, symptoms=widget_seleccion.value)
result

[{'disease': '(vertigo) paroymsal  positional vertigo', 'similarity': 1.0},
 {'disease': 'hepatitis c', 'similarity': 1.0},
 {'disease': 'gerd', 'similarity': 1.0},
 {'disease': 'hypoglycemia', 'similarity': 1.0},
 {'disease': 'impetigo', 'similarity': 1.0},
 {'disease': 'common cold', 'similarity': 1.0},
 {'disease': 'chicken pox', 'similarity': 1.0},
 {'disease': 'aids', 'similarity': 1.0},
 {'disease': 'heart attack', 'similarity': 1.0},
 {'disease': 'paralysis (brain hemorrhage)', 'similarity': 1.0},
 {'disease': 'gastroenteritis', 'similarity': 1.0},
 {'disease': 'allergy', 'similarity': 1.0},
 {'disease': 'typhoid', 'similarity': 1.0},
 {'disease': 'pneumonia', 'similarity': 1.0},
 {'disease': 'hepatitis b', 'similarity': 1.0},
 {'disease': 'chronic cholestasis', 'similarity': 1.0},
 {'disease': 'peptic ulcer diseae', 'similarity': 1.0},
 {'disease': 'alcoholic hepatitis', 'similarity': 1.0},
 {'disease': 'migraine', 'similarity': 1.0},
 {'disease': 'bronchial asthma', 'similarit